# Building the Hello World Skill
This notebook demonstrates how to use the Cortex Python SDK to build a simple Skill.  For simple cases, the entire Skill can be defined, tested, and deployed right from a notebook.

### Requires installation of cortex-python, cortex-python[builders], cortex-python[viz]

In [ ]:
# Basic setup
%run ./setup.ipynb

## Building a Cortex Action using iPython Magic
The Cortex Action is a self-contained code modeule that encapsulates a task - in other words, it is a function in computer science terms.  It can be a short running service, long running batch job, or stateful microservice.  For our simple "Hello World" example, we can create our Action using a function and leverage the __%%cortex_action__ iPython cell magic to automatically deploy it to the Cortex Cloud.

In [ ]:
%%cortex_action --name demo/hello-world-action --function hello_world --base-image c12e/cortex-python36:29c5a9c  
from cortex import Message, Cortex

def hello_world(params):
    msg = Message(params)
    text = msg.payload.get('yourName', '<no name>')
    cortex = Cortex.client(api_endpoint=msg.apiEndpoint, token=msg.token)
    return cortex.message({'message': 'Hello %s!' % text}).to_params()

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

#### If not configured through cli, use Cortex.login() to configure through notebook. Configuring through Cli is the preffered route

In [ ]:
# Instantiate Cortex Client
#Cortex.login()
cortex = Cortex.client()

# Retrieve our Action that was deployed above
action = cortex.action('demo/hello-world-action')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _text_ parameter in the Message payload.

In [ ]:
rs = action.invoke(cortex.message({'yourName': 'Cortex'}))
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Hello World" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('demo/hello-world').title('Hello World').description('Introductory Hello World Skill')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('say-hello').title('Your Name').parameter(name='yourName', type='string').all_routing(action, 'greeting').build()

In the previous step, we referenced an Output called **greeting**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('greeting').title('Greeting').parameter(name='message', type='string').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='say-hello', message=Message.with_payload({'yourName': 'Cortex'}))
rs.payload